# EEG-02 Exercises

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('notebook', font_scale=1.5)
%matplotlib inline

Today's demonstration will introduce epoching and event-related potential analysis using `mne-python`. We will inspect EEG data in response to visual and auditory stimuli. We will start by loading the data from last session.

In [ ]:
from mne.io import read_raw_fif

## Load data.
f = os.path.join('..','data','sub-01_task-audvis_preproc_raw.fif')
raw = read_raw_fif(f, preload=True, verbose=False)

## Section 1: Finding and defining events

In addition to the EEG and peripheral channels, our recording includes trigger channels. Trigger channels mark the onset/offset of events during recording. In our recording in particular, STI 014 is the trigger channel that was used for combining all the events to a single channel. It has several pulses of different amplitude throughout the recording. These pulses correspond to different stimuli presented to the subject during the acquisition. The pulses and their corresponding events are defined in the table below.

| Name   | #  | Contents                                |
|--------|----|-----------------------------------------|
| LA     | 1  | Response to left-ear auditory stimulus  |
| RA     | 2  | Response to right-ear auditory stimulus |
| LV     | 3  | Response to left visual field stimulus  |
| RV     | 4  | Response to right visual field stimulus |
| Smiley | 5  | Response to the smiley face             |
| Button | 32 | Response triggered by the button press  |

These are the events we are going to align the epochs to. To create an event list from raw data, we simply call a function dedicated just for that. Since the event list is simply a numpy array, you can also manually create one. If you create one from an outside source (like a separate file of events), pay special attention in aligning the events correctly with the raw data.

In [ ]:
from mne import find_events
from mne.viz import plot_events

## Find events.
events = find_events(raw)
print(events[:5])

The event list contains three columns. The first column corresponds to sample number. To convert this to seconds, you should divide the sample number by the used sampling frequency. The second column is reserved for the old value of the trigger channel at the time of transition, but is currently not in use. The third column is the trigger id (amplitude of the pulse).

In [ ]:
## Plot the events.
event_id = {'Auditory/Left': 1, 'Auditory/Right': 2, 'Visual/Left': 3, 'Visual/Right': 4,
            'smiley': 5, 'button': 32}
color = {1: '#1f77b4', 2: '#ff7f0e', 3: '#2ca02c', 4: '#d62728', 5: '#9467bd', 32: '#8c564b'}

fig = plot_events(events, raw.info['sfreq'], raw.first_samp, color=color, event_id=event_id);

## Section 2: Epoching

Epoching describes the process of taking snapshots of the data centered around some event of interest. We will perform epoching using the `mne.Epochs` constructor. To do so, we need to define some parameters for our epoching.

In this tutorial we are only interested in triggers 1, 2, 3 and 4. These triggers correspond to auditory and visual stimuli. The event_id here can be an int, a list of ints or a dict. With dicts it is possible to assign these ids to distinct categories.

Next we need to define the windows of interest. The values tmin and tmax refer to offsets in relation to the events. Here we make epochs that collect the data from -500 ms before to 500 ms after the event. To get some meaningful results, we also want to baseline the epochs. Baselining computes the mean over the baseline period and adjusts the data accordingly. The epochs constructor uses a baseline period from tmin to 0.0 seconds by default, but it is wise to be explicit. That way you are less likely to end up with surprises along the way. None as the first element of the tuple refers to the start of the time window (-200 ms in this case). See `mne.Epochs` for more information.



In [ ]:
from mne import Epochs, pick_types

## Define events of interest.
event_id = dict(LA=1, RA=2, LV=3, RV=4)

## Define epoch lengths.
tmin = -0.5
tmax = 0.5
baseline = (None, -0.1)

## Define channels + rejection.
picks = pick_types(raw.info, meg=False, eeg=True)
reject = dict(eeg = 100e-6)

## Perform epoching.
epochs = Epochs(raw, events, event_id=event_id, tmin=tmin, tmax=tmax, baseline=baseline,
                picks=picks, reject=reject, verbose=False)

## Drop bad epochs.
epochs.drop_bad()

fout = os.path.join('..','data','sub-01_task-audvis-epo.fif')
epochs.save(fout)

## Section 3: ERP Analysis & Visualization

Now that we have defined our epochs, we can inspect the event-related potentials. There are a great many example tutorials on visualizing evoked potentials [here](https://www.martinos.org/mne/stable/auto_tutorials/plot_visualize_evoked.html). We demonstrate a few below.

### Auditory Stimuli
First we compute the evoked response for each auditory stimulus condition.

In [ ]:
## Average within each condition.
LA_evoked = epochs['LA'].average()
RA_evoked = epochs['RA'].average()

Next let's plot the ERP for each condition.

In [ ]:
print('Left auditory')
fig, ax = plt.subplots(1,1,figsize=(12,4))
fig = LA_evoked.plot(spatial_colors=True, xlim=(-0.1,0.5), axes=ax);

print('Right auditory')
fig, ax = plt.subplots(1,1,figsize=(12,4))
fig = RA_evoked.plot(spatial_colors=True, xlim=(-0.1,0.5), axes=ax);

We can also plot the scalp topographic maps for each condition.

In [ ]:
print('Left auditory')
LA_evoked.plot_topomap(times=np.arange(0,0.35,0.05));
print('Right auditory')
RA_evoked.plot_topomap(times=np.arange(0,0.35,0.05));

Using the sensor layout from the previous notebook, we can choose a sensor that is clearly picking up a response to the auditory stimuli. **EEG 013** is a good candidate.

In [ ]:
from mne.viz import plot_compare_evokeds

## Construct evoked dictionary.
auditory = evoked_dict = dict(LA=LA_evoked.copy(), RA=RA_evoked.copy())
for k in ['LA', 'RA']: auditory[k] = auditory[k].crop(-0.1, 0.5)

## Plot.
fig = plot_compare_evokeds(auditory, picks=[epochs.ch_names.index('EEG 013')]);

### Visual Stimuli
First we compute the evoked response for each auditory stimulus condition.

In [ ]:
## Average within each condition.
LV_evoked = epochs['LV'].average()
RV_evoked = epochs['RV'].average()

Next let's plot the ERP for each condition.

In [ ]:
print('Left visual')
fig, ax = plt.subplots(1,1,figsize=(12,4))
fig = LV_evoked.plot(spatial_colors=True, xlim=(-0.1,0.5), axes=ax);

print('Right visual')
fig, ax = plt.subplots(1,1,figsize=(12,4))
fig = RV_evoked.plot(spatial_colors=True, xlim=(-0.1,0.5), axes=ax);

We can also plot the scalp topographic maps for each condition.

In [ ]:
print('Left visual')
LV_evoked.plot_topomap(times=np.arange(0.05,0.25,0.025));
print('Right visual')
RV_evoked.plot_topomap(times=np.arange(0.05,0.25,0.025));

Using the sensor layout from the previous notebook, we can choose a sensor that is clearly picking up a response to the visual stimuli. Here we are observing strong laterality, so we visual two sensors: **EEG 056** and **EEG 057**. We will clearly see that that right- and left-presented stimuli are more strongly represented in the contralateral hemispheres.

In [ ]:
## Construct evoked dictionary.
visual = evoked_dict = dict(LV=LV_evoked, RV=RV_evoked)
for k in ['LV', 'RV']: visual[k] = visual[k].crop(-0.1, 0.5)

## Plot.
plot_compare_evokeds(visual, picks=[epochs.ch_names.index('EEG 057')]);

In [ ]:
plot_compare_evokeds(visual, picks=[epochs.ch_names.index('EEG 056')]);